[<- Anterior](encoding.ipynb) | [Próximo ->](../validacao/README.md)

# Pipelines e ColumnTransformers

Pipelines e ColumnTransformers são estruturas que permitem padronizar o pré-processamento de dados e o treinamento de um modelo dentro de um único objeto, tornando a etapa de pré-processamento mais simples.

**O que será abordado**
- Imports
- Dataset de teste
- ColumnTransformers
- Pipelines
- Exemplo de uso

# Imports

In [ ]:
# Import da biblioteca Pandas -> Utilizada na visualização dos dados e criação do Dataframe
import pandas as pd

# Importação da Classe SimpleImputer -> Utilizada nas imputações de valores nulos ou ausentes
from sklearn.impute import SimpleImputer

# Importação da Classe OneHotEncoder -> Utilizada para realizar a categorização dos dados
from sklearn.preprocessing import OneHotEncoder

# Importação da função train_test_split -> Utilizada para divisão do dataset em conjunto de treino e conjunto de teste
from sklearn.model_selection import train_test_split

#Importa a classe LogisticRegression -> Modelo de Classificação que será usado na sessão de exemplo de aplicação
from sklearn.linear_model import LogisticRegression

# Dataset de teste

Para o estudo do processo de criação de pipelines será utilizado um dataset que possui diferentes tipos de dados e valores nulos. Dessa forma o dataframe de teste pode se aproximar de um caso real e pode ser usado para estudar diferentes ferramentas de pré-processamento.

O dataset utilizado está disponível na pasta Data desse diretório ([aperte aqui para acessar](../Data/PreProcessing_Test.csv)) e possui 100 linhas, apresentando valores nulos nas colunas 'Genero' e 'Salario'. O dataset apresenta uma coluna binaria 'Comprou' que indica se uma pessoa comprou um certo produto


In [ ]:
#leitura do dataset
df = pd.read_csv('../Data/pre_processing_test.csv')
df.head()

# ColumnTransformers

ColumnTransformers são objetos que indicam a ordem e quais procedimentos que devem ser realizados em um ou mais conjuntos de dados.

Para criar um column transformer é necessário criar um objeto da classe ColumnTransformer, que recebe como parâmetro um array contendo tuplas que descrevem os procedimentos que serão realizados por ele. Exemplo:   

```py
nome_do_ColumnTransformer = ColumnTransformer([   
    (nome_do_processo, função_a_ser_realizada, conjundo_de_dados_que_será_modificado)   
    (nome_do_processo2, função_a_ser_realizada2, conjundo_de_dados_que_será_modificado) #Opcional   
])
```

Veja um exemplo abaixo:

In [ ]:
#Imprta a Classe ColumnTransformer
from sklearn.compose import ColumnTransformer

#ColumnTransformer para as colunas Numericas
num_transformer = ColumnTransformer([
    ('num_imputer', SimpleImputer(strategy='mean'), ['Idade', 'Salario']), # Imputer das colunas numericas
    ('scaler', StandardScaler(), ['Idade', 'Salario']) # Scaler das colunas numericas
])

#ColumnTransformer para as colunas de Texto
str_transformer = ColumnTransformer([
    ('Imputer', SimpleImputer(strategy='most_frequent'), df[['Genero', 'Setor']]),
    ('cat', OneHotEncoder(sparse_output=False), ['Genero', 'Setor'])
])

#Alplica o ColumnTransform numerico ao dataframe 
num_cols = num_transformer.fit_transform(df)

#transformação do resultado em um dataframe: ESSA ETAPA SERVE APENAS PARA VISUALIZAR OS REUSLTADOS
num_cols_df = pd.DataFrame(num_cols, columns=num_transformer.get_feature_names_out())
num_cols_df

Perceba que, ao aplicar o transformer numérico ao dataframe o resultado é um Array que contém os dados da aplicação de cada etapa nas colunas desejadas. Exemplo:

- A primeira etapa do ColumnTansformer criado é a aplicação do `SimpleImputer()` nas colunas 'Idade' e 'Salario'. Portanto as primeiras colunas trazem o resultado dessa função, sendo que, cada uma representa o resultado em uma coluna diferente.
- A segunda etapa do ColumnTransformer criado é a aplicação do `StandarScaler()` nas colunas 'Idade' e 'Salario'. Portanto as colunas colunas trazem o resultado dessa função, sendo que, cada uma representa o resultado em uma coluna diferente.

Perceba que, ao realizar o `.fit_tranform()` em um dataframe do Pandas não é necessário informar quais colunas devem sofrer a aplicação das funções. Isso acontece por que o nome das colunas no **ColumnTransform** utilizado possuem os mesmos nomes das colunas no dataframe em que a função deve ser aplicada

# Pipelines

Pipelines são objetos que indicam em qual sequencia um conjunto de procedimentos deve ser realizado.

Para criar um pipeline é necessário criar um objeto da classe Pipeline, que recebe como parâmetro um array contendo tuplas que descrevem os procedimentos que serão realizados por ele. Exemplo:   

```py
nome_do_Pipeline = Pipeline([    
    (nome_do_processo, função_a_ser_realizada)    
    (nome_do_processo2, função_a_ser_realizada2) #Opcional    
])
```

Veja um exemplo abaixo:

In [ ]:
#Importação da Classe de Pipeline
from sklearn.pipeline import Pipeline

#Criação do Pipeline numérico
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# do pipeline de Texto
str_pipeline = num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('cat', OneHotEncoder(sparse_output=False))
])

#Aplica o Pipeline de texto ao dataframe 
str_cols = str_pipeline.fit_transform(df[['Genero', 'Setor']])

#transformação do resultado em um dataframe: ESSA ETAPA SERVE APENAS PARA VISUALIZAR OS REUSLTADOS
str_cols_df = pd.DataFrame(str_cols)
str_cols_df

Perceba que, ao aplicar o pipeline de Texto às colunas 'Genero' e 'Setor' o resultado é um Array que comtém os dados da aplicação de cada etapa nas colunas desejadas. Exemplo:

- A primeira etapa do Pipeline criado é a aplicação do `SimpleImputer()` nos dados fornecidos como parametro do método `.fit_transform()`. Portanto as primeiras colunas trazem o resultado dessa função, sendo que, cada uma representa o resultado em uma coluna diferente.
- A segunda etapa do Pipeline criado é a aplicação do `OneHotEncoder()` nos dados fornecidos como parametro do método `.fit_transform()`. Portanto as colunas trazem o resultado dessa função, sendo que, cada uma representa o resultado em uma coluna diferente.

Perceba que, ao realizar o `.fit_tranform()` é necessário informar qual conjunto de dados deve receber as funções informadas dentro do pipeline. Isso acontece por que, ao contrário do que ocorre com a classe [ColumnTransform](#columntransformers), pipelines apenas aplicam as funções informadas, sem saber de fato à quais colunas esses processos devem ser aplicados.

**ESSE CASO É APENAS UM EXEMPLO PARA ESCLARECER O FUNCIONAMENTO DE UM PIPELINE E NÃO CORRESPONDE A UM USO COMUM DESSE RECURSO.**   
**UM USO MAIS COMUM DE PIPELINES PODE SER VISTO NA SESSÃO [Exemplo de Exemplo de Uso com Pipelines + ColumnTransformers + Regressão Logistica](#exemplo-de-uso-com-pipelines--columntransformers--logisticregression)**

# Exemplo de Uso com Pipelines + ColumnTransformers + LogisticRegression

Esse tópico é dedicado a fornecer um exemplo mais clássico de como Pipelines e ColumnTransforms podem ser usados para realizar simplificar o processo de pré-processamento dos dados e treinar um modelo de Classificação.

In [ ]:
#Importa a classe LogisticRegression, que corresponde ao modelo que será treinado nessa sessão
from sklearn.linear_model import LogisticRegression

x = df.drop(columns='Comprou') # Cria uma variavel que armazena o Dataframe com os dados de ENTRADA do modelo
y = df['Comprou'] # Cria um Dataframe que corresponde aos dados de SAÍDA DESEJADA do modelo

num_cols = ['Idade', 'Salario'] # Colunas Numericas
str_cols = ['Genero', 'Idade'] # Colunas de Texto

#Criação do pipeline para o tratamento de dados numericos
num_pipeline = Pipeline([
    ('Imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

#Criação do Pipeline para o tratamento de dados de Texto
str_pipeline = Pipeline([
    ('Imputer', SimpleImputer(strategy='most_frequent')),
    ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

#Junção dos pipelines com ColumnTransform
preprocessing = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('str', str_pipeline, str_cols)
])

#Criação do Pipeline Final -> Engloba a etapa de pré-processamento e inclui um modelo de Regressão Logistica
model = Pipeline([
    ('preprocessamento', preprocessing), #Realiza o pré-processamento
    ('model', LogisticRegression()) #Modelo de Regressão Logistica
])

#Disisão dos dados em conjuto de treino e de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#Realização do pré-processamento dos dados e treino do modelo
model.fit(x_train, y_train)

#Avaliação do modelo
print(f'A precisão do modelo é: {model.score(x_test, y_test) *100}%')  

O exemplo acima apresenta uma aplicação mais pratica e completa do uso de Pipelines e ColumnTransformers.   

Embora ambos realizem um conjunto de procedimentos nos dados de entrada é possível notar que ColunTransformers só realizam esses procedimentos em colunas pré-determinadas. Além disso, um ColumnTransform pode realizar os procediemntos descritos em um Pipeline. Da mesma forma, é possível utilizar Pipeline para realizar os procedimentos descritos em um ColumnTransform.

> O processo de Disisão dos dados em conjuto de treino e de teste é melhor detalhaodo em [train_test_split.ipynb](../Validacao/train_test_split.ipynb)

#
[<- Anterior](encoding.ipynb) | [Próximo ->](../Validacao/README.md)